In [2]:
!pip install selenium

In [ ]:
!pip install --upgrade selenium

In [3]:
!pip install requests-html

In [4]:
import pandas as pd
from datetime import datetime
import requests
import time 
from requests_html import HTMLSession
from PIL import Image
from io import BytesIO

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import display, clear_output

In [5]:
s = Service('chromedriver')
driver = webdriver.Chrome(service=s)

In [6]:
driver.get("https://www.airbnb.com/?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&search_mode=flex_destinations_search&flexible_trip_lengths%5B%5D=one_week&location_search=MIN_MAP_BOUNDS&price_filter_input_type=0&category_tag=Tag%3A7769&search_type=category_change")

In [ ]:
def scroll_window():
    total_height = 33000
    for i in range(1, total_height, 700):
        driver.execute_script("window.scrollTo(0, {});".format(i))
        time.sleep(3)

In [ ]:
scroll_window()
time.sleep(3)
beach_links=driver.find_elements(by=By.CLASS_NAME,value="ln2bl2p")
print(len(beach_links))
with open("beach_urls.txt","w") as f:
    for i in beach_links:
        f.write(i.get_attribute('href'))
        f.write("\n")

# Function for Title

In [7]:
def titleHeading(driver):
    try:
        title=driver.find_element(by=By.CLASS_NAME,value="_fecoyn4").text
        print("title",title)
        return title
    except:
        print("destination not found , may be link is broken")
        return None

# Function for Rating

In [8]:
def placeRating(driver):
    try:
        rating=driver.find_element(by=By.CLASS_NAME,value="_17p6nbba").text
        print("rating",rating)
        return rating
    except:
        print("destination not found , may be link is broken")
        return None

# Function for Reviews

In [9]:
def placeTotalReviw(driver):
    try:
        reviewNumber=driver.find_element(by=By.CLASS_NAME,value="_11eqlma4").text
        print("total_reviews",reviewNumber)
        return reviewNumber
    except:
        print("destination not found , may be link is broken")
        return None

# Function for Price

In [10]:
def placeRent(driver):
    try:
        time.sleep(3)
        rent=driver.find_element(by=By.XPATH,value="//*/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[1]").text
        print("Place_rent",rent)
        return rent
    except:
        print("destination not found , may be link is broken")
        return None

# Function for Location

In [11]:
def placeLocation(driver):
    try:
        location=driver.find_element(by=By.CLASS_NAME,value="_9xiloll").text
        print("Location",location)
        return location
    except:
        print("destination not found , may be link is broken")
        return None

# Function for Description

In [12]:
def placeDescription(driver):
    try:
        if(show_more_click(driver,"desc")==True):
            time.sleep(3)
            description =WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[11]/section/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div/div/section"))).text
            driver.back()
        else:
            description=driver.find_element(by=By.XPATH,value="//*/div/div[1]/div[3]/div/div[1]/div/div[4]/div/div[2]/div/span/span").text        
#         driver.refresh()
        print("Place_description",description)
        return description
    except:
        print("destination not found , may be link is broken")
        return None

# Opening Driver

In [13]:
#open driver
def openDriver():
    op=webdriver.ChromeOptions()
    op.add_argument("headless")
    s = Service('chromedriver')
    driver = webdriver.Chrome(service=s)
    return driver

# Single Url Scrapping

In [15]:
def product_scrap(driver,url,category):
    driver.get(url)
    time.sleep(5)
    #single page data scrapping
    title=titleHeading(driver)
    #place rating
    rating=placeRating(driver)
    #place total review
    total_review=placeTotalReviw(driver)
    #place rent
    rent=placeRent(driver)
    #place location
    location=placeLocation(driver)
    #place description
    description=placeDescription(driver)
    time.sleep(2)
    return title,rating,total_review,rent,location,description


# All URLs Scrapping

In [18]:
category="destination_images"
with open("beach_urls.txt","r") as f:
    single_url=f.readlines()
    titles,ratings,total_reviews,rents,locations,descriptions=[],[],[],[],[],[]
    airbnb_scrap_partwise(0,98,titles,ratings,total_reviews,rents,locations,descriptions)
    time.sleep(30)
    airbnb_scrap_partwise(99,180,titles,ratings,total_reviews,rents,locations,descriptions)
    time.sleep(30)
    airbnb_scrap_partwise(181,224,titles,ratings,total_reviews,rents,locations,descriptions)

43 https://www.airbnb.com/rooms/26091857?adults=1&category_tag=Tag%3A7769&children=0&infants=0&search_mode=flex_destinations_search&check_in=2022-11-13&check_out=2022-11-18&previous_page_section_name=1000&federated_search_id=9f9047f1-6ca1-4808-a9a3-10c9efa502a1

title Flat 601, Rizvi Mahal, Waterfield Road, Bandra W
rating 5.0 ·
total_reviews 14 reviews
Place_rent $42
Location Mumbai, Maharashtra, India
destination not found , may be link is broken


# Liminted Number of URLs Scrapping

In [17]:
def airbnb_scrap_partwise(num1,num2,titles,ratings,total_reviews,rents,locations,descriptions):
    cnt=1
    for s in single_url[num1:num2]:
        print(cnt,s)
        driver=openDriver()
        title,rating,total_review,rent,location,description=product_scrap(driver,s,category)
        if title:
            titles.append(title)
            ratings.append(rating)
            total_reviews.append(total_review)
            rents.append(rent)
            locations.append(location)
            descriptions.append(description)
           
            
            cnt+=1
            print("=============================================")
            clear_output(wait=True)
    

# Saving Information in Text File

In [19]:
with open("Airbnb_Scrab.txt","w",encoding="utf-8") as f:
    f.write("No ,Title ,Ratings ,Total_reviews ,Rents ,Location ,Description")
    f.write("\n\n")
    for i,t,r,tr,ren,l,d in zip(range(1,len(titles)+1),titles,ratings,total_reviews,rents,locations,descriptions):
        f.write(f"{i} ,{t} ,{r} ,{tr} ,{ren} ,{l} ,{d} ")
        f.write("\n\n")